# repro-rpg のデザイン案
以下Battle Classに隠蔽されているコード
遊ぶときは
```
imoprt repro-rpg
```
とかになる


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import random
from abc import ABCMeta, abstractmethod

In [2]:
class Skill():
    
    @abstractmethod
    def __init__(self):
        self.name = ''
        self.type = 'normal'
        self.value = 1
    
    def __process__(self):
        pass

In [3]:
class NormalAttack(Skill):
    def __init__(self):
        self.name = '攻撃'
        self.type = 'normal'
        self.value = 30

    def process(self):
        pass

In [4]:
class StrongAttack(Skill):
    def __init__(self):
        self.name = '強攻撃'
        self.type = 'normal'
        self.value = 100
        
    def process(self):
        pass

In [5]:
n = NormalAttack()

In [6]:
n.type

'normal'

In [7]:
class BattleUnit():
    def __init__(self):
        self.health = 10
        self.attack = 10
        self.power = 5
        self.defence = 2
        self.speed = 5
        self.isPlayer = True
        self.skills = None

In [8]:
class Player(BattleUnit):
    
    def __init__(self, level=1):
        self.name = 'オレサマ'
        self.health = 10 + level * 0.1
        self.attack = 6 + level * 0.1
        self.power = 5 + level * 0.1
        self.defence = 2 + level * 0.1
        self.speed = 5 + level * 0.1
        self.isPlayer = True
        self.skills =[NormalAttack(), StrongAttack()]
        

In [9]:
class Enemy(BattleUnit):
    
    def __init__(self, level=1):
        self.name = '敵 lv.' + str(level)
        self.health = 10 + level * 0.2
        self.attack = 5 + level * 0.2
        self.power = 3 + level * 0.3
        self.defence = 1 + level * 0.2
        self.speed = 2 + level * 0.2
        self.isPlayer = False
        self.skills =[NormalAttack()]
    
    def algorithm(self):
        return 0

In [19]:
class Battle():
    def __init__(self):
        self.battle_units = []
        self.finished = False
        self.player_win = False
        
    def basic_algorithm(self):
        skill = NormalAttack()
        return skill
        
    def appry_unit(self, unit):
        self.battle_units.append(unit)
    
    def current_battle_info(self):
        return self.battle_units # tentative
    
    def skill_list(self):
        return [skill.name for skill in self.battle_units[0].skills]

    def apply_algorithm(self, func):
        self.algorithm = func
    
    
    def process_turn(self):
        random.shuffle(self.battle_units)
        self.battle_units = sorted(self.battle_units, key=lambda t: (t.speed), reverse=True)
        #print(self.battle_units)
        
        for self.active_unit in self.battle_units:
            used_skill = None
            if self.active_unit.isPlayer:
                index_skill = self.algorithm(self.current_battle_info())
                used_skill = self.active_unit.skills[index_skill]
                print(self.active_unit.name + 'の' + used_skill.name)
                
                # calc damage
                damage = self.active_unit.attack * used_skill.value * 0.01 - self.battle_units[1].defence
                damage = np.maximum(np.random.randint(0,2), damage)             
                self.battle_units[1].health -= damage
                print(self.battle_units[1].name+'に'+str(round(damage))+'のダメージ')
                
            else:
                index_skill = self.active_unit.algorithm()
                used_skill = self.active_unit.skills[index_skill]
                print(self.active_unit.name + 'の' + used_skill.name)
                
                # calc damage
                damage = self.active_unit.attack * used_skill.value * 0.01 - self.battle_units[0].defence
                damage = np.maximum(np.random.randint(0,2), damage)             
                self.battle_units[0].health -= damage
                print(self.battle_units[0].name+'に'+str(round(damage))+'のダメージ')
            
            
            for i in reversed(range(len(self.battle_units))):
                if self.battle_units[i].health <= 0:
                    print(self.battle_units[i].name + 'は死んだ')
                    del self.battle_units[i]
                    
            alives = [unit.isPlayer for unit in self.battle_units]
            n_players = np.count_nonzero(alives)
            n_enemies = len(alives) - n_players
            
            if n_players == 0:
                print('Game Over')
                self.finished = True
            if n_enemies == 0:
                print('Player Win!')
                self.finished = True
                self.player_win = True
                
                
    def process(self):
        
        i_turn = 1
        while not self.finished:
            print(str(i_turn) + 'ターン')
            self.process_turn()
            i_turn += 1
            if i_turn > 100:
                break

## ユーザーが工夫して書くアルゴリズム

 - current_battle_info を受け取って, 自分の使いたいSkillのindexを返す
 - 現状はcurrent_battle_infoはすべての情報を持っているが, これはBattleクラス内で情報を欠落させて渡す

In [20]:
# User should write this function, then apply battle class

def my_algorithm(current_battle_info):
    skill_list = [skill.name for skill in current_battle_info[0].skills]
    #print(skill_list)
    # -> ['攻撃', '強攻撃']
    rand = np.random.randint(0,100)
    if rand < 30:
        return 1 # return your skill index
    else:
        return 0 # return your skill index

In [38]:
# 1戦闘のスタート

player = Player()
enemy1 = Enemy(level=10)
enemy2 = Enemy(level=9)


b = Battle()
b.appry_unit(player)
b.appry_unit(enemy1)
b.appry_unit(enemy2)

In [39]:
b.apply_algorithm(my_algorithm)

In [40]:
b.process()

1ターン
オレサマの攻撃
敵 lv.10に1.0のダメージ
敵 lv.10の攻撃
オレサマに1.0のダメージ
敵 lv.9の攻撃
オレサマに0.0のダメージ
2ターン
オレサマの攻撃
敵 lv.10に0.0のダメージ
敵 lv.10の攻撃
オレサマに0.0のダメージ
敵 lv.9の攻撃
オレサマに1.0のダメージ
3ターン
オレサマの強攻撃
敵 lv.10に3.0のダメージ
敵 lv.10の攻撃
オレサマに0.0のダメージ
敵 lv.9の攻撃
オレサマに1.0のダメージ
4ターン
オレサマの攻撃
敵 lv.10に0.0のダメージ
敵 lv.10の攻撃
オレサマに0.0のダメージ
敵 lv.9の攻撃
オレサマに0.0のダメージ
5ターン
オレサマの攻撃
敵 lv.10に0.0のダメージ
敵 lv.10の攻撃
オレサマに1.0のダメージ
敵 lv.9の攻撃
オレサマに0.0のダメージ
6ターン
オレサマの攻撃
敵 lv.10に0.0のダメージ
敵 lv.10の攻撃
オレサマに0.0のダメージ
敵 lv.9の攻撃
オレサマに1.0のダメージ
7ターン
オレサマの攻撃
敵 lv.10に0.0のダメージ
敵 lv.10の攻撃
オレサマに1.0のダメージ
敵 lv.9の攻撃
オレサマに0.0のダメージ
8ターン
オレサマの強攻撃
敵 lv.10に3.0のダメージ
敵 lv.10の攻撃
オレサマに0.0のダメージ
敵 lv.9の攻撃
オレサマに0.0のダメージ
9ターン
オレサマの強攻撃
敵 lv.10に3.0のダメージ
敵 lv.10の攻撃
オレサマに0.0のダメージ
敵 lv.9の攻撃
オレサマに0.0のダメージ
10ターン
オレサマの攻撃
敵 lv.10に0.0のダメージ
敵 lv.10の攻撃
オレサマに0.0のダメージ
敵 lv.9の攻撃
オレサマに0.0のダメージ
11ターン
オレサマの強攻撃
敵 lv.10に3.0のダメージ
敵 lv.10は死んだ
敵 lv.9の攻撃
オレサマに1.0のダメージ
12ターン
オレサマの攻撃
敵 lv.9に0.0のダメージ
敵 lv.9の攻撃
オレサマに0.0のダメージ
13ターン
オレサマの攻撃
敵 lv.9に1.0のダメージ
敵 lv.9の攻撃
オレサマに0.0のダメージ
14ターン
オレサマの強攻撃
敵 lv.9に3.0のダメージ